In [1]:
import re
import time
#import itertools
from itertools import combinations, permutations, product, cycle, islice
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import plotly.graph_objects as go
import plotly.express as px
from bs4 import BeautifulSoup as bs4
from selenium.webdriver import Chrome
from selenium import webdriver
from selenium.webdriver.common.by import By

from optimize_lineup import Optimized_Lineups
from fantasy_utils import Fantasy_Projections as fp
import fantasy_utils as fu
fp = fu.Fantasy_Projections()

In [2]:
proj_url = 'https://xdl.baseball.cbssports.com/stats/stats-main'
elig_url = 'https://xdl.baseball.cbssports.com/teams/eligibility/'

In [3]:
eligibility_url_dict = {"9 Grand Kids":'30', 'Brewbirds':'3', 'Charmer':'36', 'Dirty Birds':'41', "Harveys Wallbangers":'4', 'Lil Trump & the Ivanabees':'27', 'Lima Time!':'38', 
                        'Roid Ragers':'44', 'Trouble with the Curve':'1', 'Ugly Spuds':'29', 'Wiscompton Wu-Tang':'42', 'Young Guns':'45', "Mom's Cookin":'47'}
position_priority = ['C', '2B', '3B', 'SS', 'OF', '1B', 'MI', 'CI', 'DH', 'SP', 'RP']

In [4]:
driver = webdriver.Chrome("C:\\WebDriver\\bin\\chromedriver.exe")
driver.get('https://www.cbssports.com/login?master_product=150&xurl=https%3A%2F%2Fwww.cbssports.com%2Flogin')
search_box = driver.find_element_by_id('userid')
search_box.send_keys('gostros09')
search_box = driver.find_element_by_id('password')
search_box.send_keys('Segneri9A')
search_box.submit()
time.sleep(3) # Let the user actually see something!
driver.get(proj_url)

# This clicks the Teams filter and opens it
time.sleep(3)
driver.find_element(By.XPATH, "//li[@class=' fantasyTeams dropdown selected_arrow large opt ']").click()

# This chooses the All Teams filter
time.sleep(4)
driver.find_element(By.XPATH, "//a[@aria-label='All Teams ']").click()

# This clicks on the Timeframe pulldown menu
time.sleep(3)
driver.find_element(By.XPATH, "//div[@class='pageFilter subFilters dropdown large']").click()
# This chooses the Rest of Season option
time.sleep(3)
driver.find_element(By.XPATH, "//li[@value='/stats/stats-main/restofseason']").click()

time.sleep(1)
html = driver.page_source
soup = bs4(html, 'html.parser')
pd.read_html(str(soup.find_all('table')[1:25]))

WebDriverException: Message: 'chromedriver.exe' executable needs to be in PATH. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [ ]:
html = driver.page_source
soup = bs4(html, 'html.parser')
ros = pd.read_html(str(soup.find_all('table')[0:24]))

In [ ]:
driver.quit()

In [ ]:
#driver.find_element(By.XPATH, "//a[@class='sc-1hq60lq-3 sc-1yvalmu-5 NutYB ciyBov']").click()
#driver.find_element(By.XPATH, "//ul[@class='sc-1hq60lq-1 sc-1yvalmu-1 haeRSb jkqOBN dropDownFilter']/li[5]").click()
#driver.find_element(By.XPATH, "//a[@class='sc-1hq60lq-3 sc-1yvalmu-5 sc-1yvalmu-6 hvtxhx ciyBov JLOPU']").click()

In [5]:
name_change = {"9 Gran...":'9 Grand Kids', 'Brewbi...':'Brewbirds', 'Charme...':'Charmer', 'Dirty ...':'Dirty Birds', 'Harvey...':'Harveys Wallbangers', 'Lil Tr...':'Wiscompton Wu-Tang', "Mom's ...":"Mom's Cookin", 
               'Lima T...':'Lima Time!', 'Roid R...':'Roid Ragers', 'Troubl...':'Trouble with the Curve', 'Ugly S...':'Ugly Spuds', 'Wiscom...':'Wiscompton Wu-Tang', 'Young ...':'Young Guns'}

In [6]:
team_list = ['Lima Time!', 'Ugly Spuds', 'Harveys Wallbangers', 'Roid Ragers', 'Charmer', 'Wiscompton Wu-Tang', '9 Grand Kids', 'Brewbirds', 'Trouble With The Curve', "Mom's Cookin", 'Dirty Birds', 'Young Guns'] 
stat_list = ['BA', 'HR', 'R', 'RBI', 'SB', 'ERA', 'Sv+Hld', 'SO', 'W', 'WHIP']

In [7]:
def clean_table(df, num):
    df = df.copy()
    if num % 2 == 0:
        df.columns = df.columns.droplevel().droplevel()  
    else:
        df.columns = df.columns.droplevel()
    
    df.drop(columns='Action', inplace=True)
    df = df.iloc[:df.index.stop-1]
    df['Team'] = df.Player.apply(lambda x: x.split()[-1])
    df['Pos'] = df.Player.apply(lambda x: x.split()[-3])
    df['Player'] = df.Player.apply(lambda x: ' '.join(x.split()[:-3]))
    df['Owner'] = df['Avail'].replace(name_change)
    df.loc[df['Rank']<=50, 'top_50'] = 1
    df.loc[df['Rank']<=100, 'top_100'] = 1
    df.loc[df['Rank']<=276, 'top_276'] = 1
    df.loc[df['Rank']>276, 'above_276'] = 1
    if num % 2 == 0:
        df.loc[:13, 'Start'] = 1
    else:
        df.loc[:8, 'Start'] = 1
        df.rename(columns={'K':'SO', 'BB':'BBa', 'H':'Ha'},inplace=True)
        df['Sv+Hld'] = df['S'] + df['HD']
        df['ER'] = df['INNs']*df['ERA']/9
    
    df.fillna(0,inplace=True)
    return df

In [8]:
def login():
    driver = webdriver.Chrome("C:\\ProgramData\\Anaconda3\\WebDriver\\bin\\chromedriver.exe")
    driver.get('https://www.cbssports.com/login?master_product=150&xurl=https%3A%2F%2Fwww.cbssports.com%2Flogin')
    search_box = driver.find_element_by_id('userid')
    search_box.send_keys('gostros09')
    search_box = driver.find_element_by_id('password')
    search_box.send_keys('Segneri9A')
    search_box.submit()
    return driver

In [9]:
def process_eligibility(val, driver):
    driver.get(elig_url+val)
    time.sleep(2.2)
    html = driver.page_source
    soup = bs4(html, 'html.parser')
    elig = pd.read_html(str(soup.find_all('table')))[0]
    elig = elig.iloc[:-1,:]
    player_ids = get_cbsid_from_eligibility(soup)
    elig['cbsid'] = player_ids
    elig['Player'] = elig['Player'].apply(lambda x: x[:re.search(r'RP|SP|1B|2B|3B|SS|LF|CF|RF|DH|C\s', x).span()[0]])
    elig.fillna(0,inplace=True)
    elig.loc[:,1:] = elig.iloc[:,1:].astype('int32')
    return elig

In [10]:
def get_cbsid_from_eligibility(s):
    player_ids = []

    # Find all the 'a' tags (links) within the table
    links = s.find('table').find_all('a')

    # Iterate over links
    for link in links:
        # Get the href attribute value
        href = link.get('href')
        # Extract the number at the end using regular expressions
        try:
            player_id = re.search(r'/(\d+)$', href).group(1)
            if int(player_id) not in player_ids:
                player_ids.append(int(player_id))
        except:
            pass
    
    return player_ids


In [11]:
def scrape_proj(driver):
    time.sleep(2)
    driver.get(proj_url)

    # This clicks the Teams filter and opens it
    time.sleep(3)
    driver.find_element(By.XPATH, "//li[@class=' fantasyTeams dropdown selected_arrow large opt ']").click()

    # This chooses the All Teams filter
    time.sleep(4)
    driver.find_element(By.XPATH, "//a[@aria-label='All Teams ']").click()

    time.sleep(3)
    html = driver.page_source
    soup = bs4(html, 'html.parser')
    cur_proj = pd.read_html(str(soup.find_all('table')[0:24]))

    # This clicks on the Timeframe pulldown menu
    time.sleep(4)
    driver.find_element(By.XPATH, "//div[@class='pageFilter subFilters dropdown large']").click()
    # This chooses the Rest of Season option
    time.sleep(4)
    driver.find_element(By.XPATH, "//li[@value='/stats/stats-main/restofseason']").click()

    time.sleep(5)
    html = driver.page_source
    soup = bs4(html, 'html.parser')
    ros_proj = pd.read_html(str(soup.find_all('table')[0:24]))
    return cur_proj, ros_proj


In [12]:
def parse_projections_data(master):
    df = pd.DataFrame()
    for n in range(len(master)):
        df = pd.concat([df, clean_table(master[n], n)])
    
    df.fillna(0,inplace=True)
    df.rename(columns={'AVG':'BA', 'INNs':'IP'}, inplace=True)
    df.drop(columns=['1B', '2B', '3B'],inplace=True)
    df = df.reset_index()
    return df

In [13]:
driver = login()

In [14]:
elig = pd.DataFrame()

for key, val in eligibility_url_dict.items():
    print(key)
    try:
        elig = pd.concat([elig, process_eligibility(val, driver)])
    except:
        print('...skipped')

9 Grand Kids


C:\Users\pddnh\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexing.py:719: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  indexer = self._get_setitem_indexer(key)


Brewbirds
Charmer
Dirty Birds
Harveys Wallbangers
Lil Trump & the Ivanabees
...skipped
Lima Time!
Roid Ragers
Trouble with the Curve
Ugly Spuds
Wiscompton Wu-Tang
Young Guns
Mom's Cookin


In [15]:
elig

,Player,C,1B,2B,3B,SS,MI,CI,OF,DH,SP,RP,cbsid
0,Ketel Marte,0,0,13,0,1,14,0,0,4,0,0,1963334
1,Framber Valdez,0,0,0,0,0,0,0,0,0,2,0,2739968
2,Pete Fairbanks,0,0,0,0,0,0,0,0,0,0,6,3117421
3,Jose Caballero,0,0,0,0,13,13,0,0,0,0,0,2836680
4,Ozzie Albies,0,0,14,0,0,14,0,0,0,0,0,2167464
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28,Nestor Cortes,0,0,0,0,0,0,0,0,0,4,0,2822128
29,Corey Seager,0,0,0,0,13,13,0,0,1,0,0,2041873
30,DJ LeMahieu,0,0,0,0,0,0,0,0,0,0,0,1740958
31,Anthony Santander,0,0,0,0,0,0,0,13,2,0,0,2504134


In [16]:
elig.to_csv('data\\2024-eligibility.csv', index=False)

In [17]:
cur_proj, ros_proj = scrape_proj(driver)

In [24]:
data = ros_proj[0]
data.Action.apply(lambda txt: -1 if len(re.findall(r'\b\d{5,}\b', txt))==0 else re.findall(r'\b\d{5,}\b', txt)[0])

AttributeError: 'DataFrame' object has no attribute 'Action'

In [22]:
#driver.find_element(By.XPATH, "//div[@class='pageFilter subFilters dropdown large']").click()
# This chooses the Rest of Season option
#time.sleep(3)
#driver.find_element(By.XPATH, "//li[@value='/stats/stats-main/restofseason']").click()

html = driver.page_source
soup = bs4(html, 'html.parser')
master = pd.read_html(str(soup.find_all('table')[0:24]))

In [23]:
master

[   9 Grand Kids                                                              \
         Batters                                                               
          Action      Avail                         Player    AB     R     H   
 0           NaN  9 Gran...           Trea Turner SS | PHI   540    87   162   
 1           NaN  9 Gran...          Ozzie Albies 2B | ATL   499    87   143   
 2           NaN  9 Gran...           Ketel Marte 2B | ARI   493    85   144   
 3           NaN  9 Gran...       Lourdes Gurriel OF | ARI   451    61   129   
 4           NaN  9 Gran...          J.T. Realmuto C | PHI   439    70   121   
 5           NaN  9 Gran...       Salvador Perez 1B,C | KC   430    55   125   
 6           NaN  9 Gran...        Chas McCormick OF | HOU   408    58   117   
 7           NaN  9 Gran...       Michael Conforto OF | SF   425    64   116   
 8           NaN  9 Gran...           Steven Kwan OF | CLE   448    76   136   
 9           NaN  9 Gran...       Edward

In [130]:
df = parse_projections_data(ros_proj)

In [131]:
#elig = pd.read_csv('data/2024-eligibility.csv')
df = df.merge(elig, on='Player', how='left')
df.loc[(df['Pos'].str.contains('OF')) & (df['OF']<5), 'OF'] = 5
df.loc[(df['Pos'].str.contains('1B')), '1B'] = 5
df.loc[(df['Pos'].str.contains('2B')), '2B'] = 5
df.loc[(df['Pos'].str.contains('3B')), '3B'] = 5
df.loc[(df['Pos'].str.contains('SS')), 'SS'] = 5
df.loc[(df['Pos'].str.contains('C')), 'C'] = 5
df.loc[(df['Pos'].str.contains('P')), 'P'] = 5
df.loc[(df['1B']>=5) | (df['3B']>=5), 'CI'] = 5
df.loc[(df['2B']>=5) | (df['SS']>=5), 'MI'] = 5
df.loc[(df['SP']>=5) | (df['RP']>=5) | (df['P']>=5), 'type'] = 'p'
df['type'].fillna('h',inplace=True)
df.loc[(df['Player']=='Shohei Ohtani') & (df['AB']>=1), 'type'] = 'h'
df.loc[df['type']=='h', 'DH'] = 5
df['DH'].fillna(0,inplace=True)
df['P'].fillna(0,inplace=True)
df['all_pos'] = df.apply(lambda x: stitch_positions(x), axis=1)

In [132]:
h_mask = (df['Start']==1) & (df['AB']>0) #(df['Pos'].isin(['C', '1B', '2B', '3B', 'LF', 'CF', 'RF', 'SS', 'DH']))
hitters = (df['AB']>0) #(df['Pos'].isin(['C', '1B', '2B', '3B', 'LF', 'CF', 'RF', 'SS', 'DH']))
p_mask = (df['Start']==1) & (df['AB']==0) #(df['Pos'].isin(['SP', 'RP']))
pitchers = (df['AB']==0) #(df['Pos'].isin(['SP', 'RP']))

df.loc[hitters, 'DH'] = 5
df.loc[pitchers, 'P'] = 5
df['P'].fillna(0,inplace=True)

df.loc[(df['SP']>=5) | (df['RP']>=5) | (df['P']>=5), 'type'] = 'p'
df['type'].fillna('h',inplace=True)
df.loc[(df['Player']=='Shohei Ohtani') & (df['AB']>10), 'type'] = 'h'

In [133]:
lgBA = df[h_mask]['H'].sum()/df[h_mask]['AB'].sum()
df.loc[hitters, 'zlgBA'] = (df[hitters]['H'] - (df[hitters]['AB'] * lgBA))

lgERA = df[p_mask]['ER'].sum()/df[p_mask]['IP'].sum()*9
df.loc[pitchers, 'zlgERA'] = (((df[pitchers]['ER']*9) - (df[pitchers]['IP']*lgERA))*-1)

lgWHIP = (df[p_mask]['BBa'].sum()+df[p_mask]['Ha'].sum())/df[p_mask]['IP'].sum()
df.loc[pitchers, 'zlgWHIP'] = (((df.loc[pitchers]['Ha']+df.loc[pitchers]['BBa'])-(df.loc[pitchers]['IP']*lgWHIP))*-1)

In [134]:
q = df[h_mask][['HR', 'RBI', 'R', 'SB', 'H', 'AB']].describe().T[['mean', 'std']].T.to_dict()
q['BA'] = {'mean':df[h_mask]['H'].sum()/df[h_mask]['AB'].sum(), 'std':df[h_mask]['BA'].std()}
q.update(df[p_mask][['W', 'SO', 'Sv+Hld', 'IP', 'ER', 'BBa', 'Ha']].describe().T[['mean', 'std']].T.to_dict())
q['ERA'] = {'mean':(df[p_mask]['ER'].sum()/df[p_mask]['IP'].sum())*9, 'std':(df[p_mask]['ER']/df[p_mask]['IP']*9).std()}
q['WHIP'] = {'mean':(df[p_mask]['BBa'].sum()+df[p_mask]['Ha'].sum())/df[p_mask]['IP'].sum(), 'std':((df[p_mask]['BBa']+df[p_mask]['Ha'])/df[p_mask]['IP']).std()}
q['zlgBA'] = {'mean':df[h_mask]['zlgBA'].mean(), 'std':df[h_mask]['zlgBA'].std()}
q['zlgERA'] = {'mean':df[p_mask]['zlgERA'].mean(), 'std':df[p_mask]['zlgERA'].std()}
q['zlgWHIP'] = {'mean':df[p_mask]['zlgWHIP'].mean(), 'std':df[p_mask]['zlgWHIP'].std()}
q

{'HR': {'mean': 21.785714285714285, 'std': 7.452960555619963},
 'RBI': {'mean': 74.02380952380952, 'std': 16.461150919429784},
 'R': {'mean': 75.50595238095238, 'std': 15.794335763219406},
 'SB': {'mean': 12.178571428571429, 'std': 10.758270139296418},
 'H': {'mean': 135.8452380952381, 'std': 22.702446014312898},
 'AB': {'mean': 493.4047619047619, 'std': 60.960302985325875},
 'BA': {'mean': 0.27532210587270184, 'std': 0.02044780610361654},
 'W': {'mean': 8.935185185185185, 'std': 4.606523509534129},
 'SO': {'mean': 144.71296296296296, 'std': 43.65969738999036},
 'Sv+Hld': {'mean': 7.824074074074074, 'std': 14.572941706318954},
 'IP': {'mean': 136.00925925925927, 'std': 45.60691359509515},
 'ER': {'mean': 51.83802469135802, 'std': 20.399451133258715},
 'BBa': {'mean': 40.583333333333336, 'std': 13.139172366145814},
 'Ha': {'mean': 119.83333333333333, 'std': 45.795053399770715},
 'ERA': {'mean': 3.43022397712574, 'std': 0.5226105915162981},
 'WHIP': {'mean': 1.1794540132071618, 'std': 0.

In [135]:
for stat in ['R', 'HR', 'RBI', 'SB', 'zlgBA']:
    df.loc[hitters, 'z'+stat] = df[hitters].apply(lambda row: fp.big_board(row, stat, q), axis=1)

df.loc[hitters & (df['zlgBA'].isna()), 'zlgBA'] = df['zlgBA'].min()-.01
df.loc[hitters & (df['zzlgBA'].isna()), 'zzlgBA'] = df['zzlgBA'].min()-.01
df.loc[hitters, 'BIGAAh'] = df[hitters]['zR'] + df[hitters]['zRBI'] + df[hitters]['zHR'] + df[hitters]['zSB'] + df[hitters]['zzlgBA']

In [136]:
for stat in ['W', 'SO', 'Sv+Hld', 'zlgERA', 'zlgWHIP']:
    df.loc[pitchers, 'z'+stat] = df[pitchers].apply(lambda row: fp.big_board(row, stat, q), axis=1)

df.loc[pitchers & (df['zzlgERA'].isna()), 'zzlgERA'] = df['zzlgERA'].min()-.01
df.loc[pitchers & (df['zzlgWHIP'].isna()), 'zzlgWHIP'] = df['zzlgWHIP'].min()-.01
df.loc[pitchers, 'BIGAAp'] = df[pitchers]['zW']+df[pitchers]['zSO']+df[pitchers]['zSv+Hld']+df[pitchers]['zzlgERA']+df[pitchers]['zzlgWHIP']

In [137]:
df['BIGAAh'].fillna(0,inplace=True)
df['BIGAAp'].fillna(0,inplace=True)
df['z'] = df['BIGAAh']+df['BIGAAp']

In [138]:
ohtani_idx = df[df['Player']=='Shohei Ohtani'].index
df.loc[ohtani_idx[0], 'z'] = df.loc[ohtani_idx[0],'BIGAAh'] + 0
df.loc[ohtani_idx[1], 'z'] = 0 + df.loc[ohtani_idx[1],'BIGAAp']
df.loc[ohtani_idx[1], ['AB', 'R', 'H', 'HR', 'RBI', 'BB', 'K', 'SB', 'CS', 'BA', 'OBP', 'SLG', 'zlgBA', 'zR', 'zHR', 'zRBI', 'zSB', 'zzlgBA', 'BIGAAh']] = df.loc[ohtani_idx[0]][['AB', 'R', 'H', 'HR', 'RBI', 'BB', 'K', 'SB', 'CS', 'BA', 'OBP', 'SLG', 'zlgBA', 'zR', 'zHR', 'zRBI', 'zSB', 'zzlgBA', 'BIGAAh']]
df = df.loc[df.index!=ohtani_idx[0]].copy()

In [139]:
df[df['Start']==1].groupby('Owner')['z'].sum().sort_values()

Owner
Harveys Wallbangers      -20.727924
Dirty Birds               -6.495915
Young Guns                -6.173900
Lima Time!                -5.206668
Mom's Cookin              -2.611557
Trouble with the Curve    -1.263711
Ugly Spuds                -1.225481
Charmer                   -0.172944
Wiscompton Wu-Tang         4.320186
Brewbirds                  7.272077
Roid Ragers               10.475725
9 Grand Kids              11.642657
Name: z, dtype: float64

In [140]:
def stitch_positions(row):
    pos_code = row[position_priority+['P']]>=5
    return list(pos_code[pos_code].index)

In [141]:
df['all_pos'] = df.apply(lambda x: stitch_positions(x), axis=1)

In [142]:
#df.to_csv('data/2024-period-1-projections.csv')
df.to_csv('data/2024-period-1-ros-projections.csv')


In [ ]:
class Optimized_Lineups:
    def __init__(self, owner, data):
        self.owner = owner
        self.data = data
        self.d = self.data[self.data['Owner']==owner][['Player','all_pos', 'z', 'type']].set_index('Player').to_dict(orient='index')      
        self.p_dict = {k:v for (k,v) in self.d.items() if 'p' in v['type']}
        self.h_dict = {k:v for (k,v) in self.d.items() if 'h' in v['type']}
        self.catchers = [k for k,v in self.h_dict.items() if 'C' in v['all_pos']]
        self.first_basemen = [k for k,v in self.h_dict.items() if '1B' in v['all_pos']]
        self.second_basemen = [k for k,v in self.h_dict.items() if '2B' in v['all_pos']]
        self.third_basemen = [k for k,v in self.h_dict.items() if '3B' in v['all_pos']]
        self.shortstops = [k for k,v in self.h_dict.items() if 'SS' in v['all_pos']]
        self.middles = [k for k,v in self.h_dict.items() if 'MI' in v['all_pos']]
        self.corners = [k for k,v in self.h_dict.items() if 'CI' in v['all_pos']]
        self.outfielders = [k for k,v in self.h_dict.items() if 'OF' in v['all_pos']]
        self.dhs = [k for k,v in self.h_dict.items() if 'DH' in v['all_pos']]
        self.ofs = [i for i in combinations(self.outfielders, 5)]
        self.dh2 = [i for i in combinations(self.dhs, 2)]
        
    def _make_pitcher_combos(self):
        self.pitcher_combos = [i for i in combinations(self.p_dict.keys(), 9)]
        self.pitcher_z_list = self._z_list(self.pitcher_combos, self.p_dict)
        self.pitcher_idx = np.argmax(self.pitcher_z_list)
        self.pitcher_optimized_z = self.pitcher_z_list[self.pitcher_idx]
        self.pitcher_optimized_lineup = self.pitcher_combos[self.pitcher_idx]
        return
    
    def _z_list(self, pos_combos, player_dict):
        sum_z = 0
        z_list = []
        for i in range(len(pos_combos)):
            for name in pos_combos[i]:
                #print(i, sum_z)
                sum_z += player_dict[name]['z']
            z_list.append(sum_z)
            sum_z = 0
        return z_list
    
    def _make_hitter_combos(self):
        inf = [i for i in product(self.catchers, self.first_basemen, self.second_basemen, self.shortstops, self.third_basemen, self.middles, self.corners, self.ofs)]
        _list = []
        for num in range(len(inf)):
            _list.append([item for item in inf[num] if type(item)!=tuple]+[item for sublist in inf[num] for item in sublist if type(sublist)==tuple])
        
        _list2 = [i for i in _list if len(set(i))==12]
        inf2 = [i for i in product(_list2, self.dh2)]
        
        _list3 = []
        for num in range(len(inf2)):
            _list3.append([list(i) for i in inf2[num] if type(i)!=tuple][0]+[list(i) for i in inf2[num] if type(i)==tuple][0])
    
        self.hitter_combos = [i for i in _list3 if len(set(i))==14]
        print(len(self.hitter_combos))
        self.hitter_z_list = self._z_list(self.hitter_combos, self.h_dict)
        print(len(self.hitter_z_list))
        self.hitter_idx = np.argmax(self.hitter_z_list)
        self.hitters_optimized_z = self.hitter_z_list[self.hitter_idx]
        self.hitters_optimized_lineup = self.hitter_combos[self.hitter_idx]
        return    
    

In [ ]:
opt_dict = {}
for t in name_change.values():
    print(t)
    opt_dict[t] = Optimized_Lineups(t, df)
    opt_dict[t]._make_pitcher_combos()
    opt_dict[t]._make_hitter_combos()
    print(f'Offense: {opt_dict[t].hitters_optimized_z}')
    print(f'Defense: {opt_dict[t].pitcher_optimized_z}')
    print(f'Total z: {opt_dict[t].pitcher_optimized_z + opt_dict[t].hitters_optimized_z}')

In [ ]:
#df = pd.read_csv('optimized_lineup_data.csv')

In [ ]:
driver.quit()

In [ ]:
df.to_csv('optimized_lineup_data2.csv')

In [ ]:
position_priority = ['C', '2B', '3B', 'SS', 'OF', '1B', 'MI', 'CI', 'DH', 'SP', 'RP']
optimized = {
    'Madness':{'C':[-1], '2B':[-1], '3B':[-1], 'OF':[-1,-1,-1,-1,-1], 'SS':[-1], '1B':[-1], 'MI':[-1], 'CI':[-1], 'DH':[-1,-1], 'SP':[-1,-1,-1,-1,-1,-1,-1,-1,-1], 'RP':[-1,-1,-1,-1,-1,-1],
                         'P':[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1], 'B':[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]},
    'Brewbirds':{'C':[-1], '2B':[-1], '3B':[-1], 'OF':[-1,-1,-1,-1,-1], 'SS':[-1], '1B':[-1], 'MI':[-1], 'CI':[-1], 'DH':[-1,-1], 'SP':[-1,-1,-1,-1,-1,-1,-1,-1,-1], 'RP':[-1,-1,-1,-1,-1,-1],
                         'P':[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1], 'B':[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]},
    'Charmer':{'C':[-1], '2B':[-1], '3B':[-1], 'OF':[-1,-1,-1,-1,-1], 'SS':[-1], '1B':[-1], 'MI':[-1], 'CI':[-1], 'DH':[-1,-1], 'SP':[-1,-1,-1,-1,-1,-1,-1,-1,-1], 'RP':[-1,-1,-1,-1,-1,-1],
                         'P':[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1], 'B':[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]},
    'Dirty Birds':{'C':[-1], '2B':[-1], '3B':[-1], 'OF':[-1,-1,-1,-1,-1], 'SS':[-1], '1B':[-1], 'MI':[-1], 'CI':[-1], 'DH':[-1,-1], 'SP':[-1,-1,-1,-1,-1,-1,-1,-1,-1], 'RP':[-1,-1,-1,-1,-1,-1],
                         'P':[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1], 'B':[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]},
    'Harvey':{'C':[-1], '2B':[-1], '3B':[-1], 'OF':[-1,-1,-1,-1,-1], 'SS':[-1], '1B':[-1], 'MI':[-1], 'CI':[-1], 'DH':[-1,-1], 'SP':[-1,-1,-1,-1,-1,-1,-1,-1,-1], 'RP':[-1,-1,-1,-1,-1,-1],
                         'P':[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1], 'B':[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]},
    'Lil Trump':{'C':[-1], '2B':[-1], '3B':[-1], 'OF':[-1,-1,-1,-1,-1], 'SS':[-1], '1B':[-1], 'MI':[-1], 'CI':[-1], 'DH':[-1,-1], 'SP':[-1,-1,-1,-1,-1,-1,-1,-1,-1], 'RP':[-1,-1,-1,-1,-1,-1],
                         'P':[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1], 'B':[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]},
    'Lima Time':{'C':[-1], '2B':[-1], '3B':[-1], 'OF':[-1,-1,-1,-1,-1], 'SS':[-1], '1B':[-1], 'MI':[-1], 'CI':[-1], 'DH':[-1,-1], 'SP':[-1,-1,-1,-1,-1,-1,-1,-1,-1], 'RP':[-1,-1,-1,-1,-1,-1],
                         'P':[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1], 'B':[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]},
    'Roiders':{'C':[-1], '2B':[-1], '3B':[-1], 'OF':[-1,-1,-1,-1,-1], 'SS':[-1], '1B':[-1], 'MI':[-1], 'CI':[-1], 'DH':[-1,-1], 'SP':[-1,-1,-1,-1,-1,-1,-1,-1,-1], 'RP':[-1,-1,-1,-1,-1,-1],
                         'P':[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1], 'B':[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]},
    'Trouble':{'C':[-1], '2B':[-1], '3B':[-1], 'OF':[-1,-1,-1,-1,-1], 'SS':[-1], '1B':[-1], 'MI':[-1], 'CI':[-1], 'DH':[-1,-1], 'SP':[-1,-1,-1,-1,-1,-1,-1,-1,-1], 'RP':[-1,-1,-1,-1,-1,-1],
                         'P':[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1], 'B':[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]},
    'Ugly Spuds':{'C':[-1], '2B':[-1], '3B':[-1], 'OF':[-1,-1,-1,-1,-1], 'SS':[-1], '1B':[-1], 'MI':[-1], 'CI':[-1], 'DH':[-1,-1], 'SP':[-1,-1,-1,-1,-1,-1,-1,-1,-1], 'RP':[-1,-1,-1,-1,-1,-1],
                         'P':[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1], 'B':[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]},
    'Wu Tang':{'C':[-1], '2B':[-1], '3B':[-1], 'OF':[-1,-1,-1,-1,-1], 'SS':[-1], '1B':[-1], 'MI':[-1], 'CI':[-1], 'DH':[-1,-1], 'SP':[-1,-1,-1,-1,-1,-1,-1,-1,-1], 'RP':[-1,-1,-1,-1,-1,-1],
                         'P':[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1], 'B':[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]},
    'Young Guns':{'C':[-1], '2B':[-1], '3B':[-1], 'OF':[-1,-1,-1,-1,-1], 'SS':[-1], '1B':[-1], 'MI':[-1], 'CI':[-1], 'DH':[-1,-1], 'SP':[-1,-1,-1,-1,-1,-1,-1,-1,-1], 'RP':[-1,-1,-1,-1,-1,-1],
                         'P':[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1], 'B':[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]},
            }
optimized

In [ ]:
for tm in df.Owner.unique():
    df.loc[df['Owner']==tm, 'opt'] = 'B'
    for df_idx, row in df[df['Owner']==tm].sort_values('z', ascending=False).iterrows():
        pos_code = df.loc[df_idx][position_priority+['P']]>=5
        #print(df_idx, row['Player'], pos_code[pos_code].index[0])

        for player_pos_val in pos_code[pos_code].index: #position list like ['SP', 'RP']
            if df.loc[df_idx, 'opt'] == 'B':
                for optimized_pos_idx in range(len(optimized[tm][player_pos_val])):
                    if optimized[tm][player_pos_val][optimized_pos_idx] == -1:
                        optimized[tm][player_pos_val][optimized_pos_idx] = row['Player']
                        df.loc[df_idx, 'opt'] = player_pos_val
                        break
                    #else: tries the next position in the optimizer position list        


In [ ]:
pd.DataFrame(optimized)

In [ ]:
for t in name_change.values():
    print(f'{t} z: {opt_dict[t].hitters_optimized_z + opt_dict[t].pitcher_optimized_z}')
    df.loc[df['Player'].isin(opt_dict[t].hitters_optimized_lineup), 'opt'] = 1
    df.loc[df['Player'].isin(opt_dict[t].pitcher_optimized_lineup), 'opt'] = 1

In [ ]:
def total_stats(row):
    #return row['H'].sum() / row['AB'].sum(), row['HR'].sum(), row['R'].sum(), row['RBI'].sum(), row['SB'].sum()
    return row['W'].sum(), row['Sv+Hld'].sum(), row['SO'].sum(), round(row['ER'].sum()/row['IP'].sum()*9,2), round((row['Ha'].sum()+row['BBa'].sum())/row['IP'].sum(),2)

In [ ]:
tm_proj = df[df['opt']==1].groupby('Owner')[['H', 'AB', 'Ha', 'BBa', 'ER', 'IP', 'R', 'RBI', 'HR', 'SB', 'W', 'SO', 'Sv+Hld']].sum()#.apply(lambda x: total_stats(x))
tm_proj['BA'] = tm_proj['H']/tm_proj['AB']
tm_proj['ERA'] = round(tm_proj['ER']/tm_proj['IP']*9,2)
tm_proj['WHIP'] = round((tm_proj['Ha']+tm_proj['BBa'])/tm_proj['IP'],2)
tm_proj

In [ ]:
for s in stat_list:
    #print(s)
    tm_proj[s+'_rank'] = tm_proj[s].rank()

tm_proj['Pts_rank'] = tm_proj.loc[:,tm_proj.columns.str.contains('_rank')].sum(axis=1)

In [ ]:
tm_proj.loc[:,tm_proj.columns.str.contains('_rank')].sort_values('Pts_rank', ascending=False)

In [ ]:
ol = Optimized_Lineups('Lima Time', df)
ol._make_pitcher_combos()
print(len(ol.pitcher_combos))
#ol.catchers
ol._make_hitter_combos()

In [ ]:
ol.pitcher_optimized_z, ol.pitcher_optimized_lineup

In [ ]:
ol.hitters_optimized_z, ol.hitters_optimized_lineup

In [ ]:
(df[df['Owner']=='Harvey'][position_priority]>=5).sum()

In [ ]:
df[df['Player'].isin(list(hitter_combos[np.argmax(hitter_z_list)]))][['BA', 'HR', 'SB', 'R', 'RBI']]#.sum()

In [ ]:
df[df['Player'].isin(list(pitcher_combos[np.argmax(pitcher_z_list)]))][['ERA', 'WHIP', 'W', 'SO', 'Sv+Hld']]#.sum()

In [ ]:
hitter_z_list[np.argmax(hitter_z_list)]+pitcher_z_list[np.argmax(pitcher_z_list)]

In [ ]:
df[(df['Owner']=='Lima Time') & (df['Start']==1)]
df[(df['Owner']=='Lima Time')][['Player', 'z', 'Sv+Hld']]#.z.sum()

In [ ]:
ranks = df.groupby('Owner').agg({'top_50':'sum', 'top_100':'sum', 'top_276':'sum', 'above_276':'sum'}).sort_values('top_276', ascending=False).reset_index()
ranks['avg'] = ranks.iloc[:,1:4].mean(axis=1)
ranks

In [ ]:
stats = df[(df['Start']==1) & (df['Pos'].isin(['1B', '2B', '3B', 'SS', 'C', 'OF', 'DH']))].groupby('Owner')[['H', 'AB', 'HR', 'SB', 'R', 'RBI', 'W', 'K', 'Sv+Hld']].sum().sort_values('K', ascending=False)
stats['BA'] = stats['H']/stats['AB']

In [ ]:
stats